# Use pyrosetta to get the per residue SAP

In [25]:
import pyrosetta
import pandas as pd

In [26]:
flags = f"""
-corrections::beta_nov16 True
"""
pyrosetta.init(flags)

┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python310.Release 2025.42+release.62f6a3258646b8927d40ecaca1860b284bd81c73 2025-10-10T13:25:42] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.ubuntu.cxx11th

In [27]:
pose = pyrosetta.io.pose_from_file(filename="./7uc5_B_renum.pdb")

core.import_pose.import_pose: File './7uc5_B_renum.pdb' automatically determined to be of type PDB from contents.
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue MET:CtermProteinFull 99
core.conformation.Conformation: Found disulfide between residues 25 80


In [28]:
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="sfxn_design" weights="beta_nov16" >
            <Reweight scoretype="sap_constraint" weight="1.0" />
        </ScoreFunction>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
        <Chain name="chainA" chains="B"/>
        

        <StoredResidueSubset name="stored_bad_sap" subset_name="bad_sap_stored" />
        <Not name="not_stored_bad_sap" selector="stored_bad_sap" />

    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>

        <OperateOnResidueSubset name="restrict_not_bad_sap" selector="not_stored_bad_sap">
            <RestrictToRepackingRLT/>
        </OperateOnResidueSubset>

    </TASKOPERATIONS>
    <MOVERS>

        <AddSapConstraintMover name="add_sap" speed="lightning" sap_goal="0"  penalty_per_sap="10" score_selector="chainA" sap_calculate_selector="chainA"  packing_correction="0" /> 

        <ClearConstraintsMover name="clear_constraints" />

    </MOVERS>
    <SIMPLE_METRICS>

        <SapScoreMetric name="my_sap_score" />

        <PerResidueSapScoreMetric name="my_per_res_sap" />

    </SIMPLE_METRICS>
    <RESIDUE_SELECTORS>

        <SimpleMetricSelector name="bad_sap" metric="my_per_res_sap" lower_bound="1.4" />

    </RESIDUE_SELECTORS>
    <MOVERS>

        <StoreResidueSubset name="store_bad_sap" subset_name="bad_sap_stored" residue_selector="bad_sap" overwrite="1" />

    </MOVERS>
    <FILTERS>

        <Time name="timed"/>
        <TrueFilter name="my_true_filter" />
        
    </FILTERS>
    <MOVERS>

        <PackRotamersMover name="redesign_bad_sap" scorefxn="sfxn_design" task_operations="restrict_not_bad_sap" />

    </MOVERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>

        <Add metrics="my_per_res_sap" />



    </PROTOCOLS>
    <OUTPUT />
</ROSETTASCRIPTS>
""").get_mover("ParsedProtocol")

protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="sfxn_design" weights="beta_nov16">
			<Reweight scoretype="sap_constraint" weight="1.0"/>
		</ScoreFunction>
	</SCOREFXNS>
	<RESIDUE_SELECTORS>
		<Chain chains="B" name="chainA"/>
		<StoredResidueSubset name="stored_bad_sap" subset_name="bad_sap_stored"/>
		<Not name="not_stored_bad_sap" selector="stored_bad_sap"/>
	</RESIDUE_SELECTORS>
	<TASKOPERATIONS>
		<OperateOnResidueSubset name="restrict_not_bad_sap" selector="not_stored_bad_sap">
			<RestrictToRepackingRLT/>
		</OperateOnResidueSubset>
	</TASKOPERATIONS>
	<MOVERS>
		<AddSapConstraintMover name="add_sap" packing_correction="0" penalty_per_sap="10" sap_calculate_selector="chainA" sap_goal="0" score_selector="chainA" speed="lightning"/>
		<ClearConstraintsMover name="clear_constraints"/>


In [29]:
test_pose = pose.clone()

In [30]:
xml.apply(test_pose)

protocols.rosetta_scripts.ParsedProtocol: =======================BEGIN METRIC my_per_res_sap=======================
protocols.rosetta_scripts.ParsedProtocol: =======================END METRIC my_per_res_sap=======================
protocols.rosetta_scripts.ParsedProtocol: setting status to success


In [31]:
test_pose.dump_pdb("7uc5_B_renum_sap.pdb")

True